In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [4]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [5]:
#vocab

In [6]:
#word_to_ix

In [7]:
len(vocab)

97

In [8]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [9]:
NGramLanguageModeler

__main__.NGramLanguageModeler

In [10]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [11]:
model

NGramLanguageModeler(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)

In [12]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[522.3496654033661, 519.7861683368683, 517.240710735321, 514.712126493454, 512.1993186473846, 509.70251655578613, 507.21931195259094, 504.7482941150665, 502.28963017463684, 499.8412735462189]


In [13]:
[word_to_ix[target]]

[64]

## CBOW

In [14]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

# create your model and train.  here are some functions to help you make
# the data ready for use by your module

In [15]:
word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
    
ix_to_word = {ix:word for word,ix in word_to_ix.items()}

In [16]:
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [17]:
context

['the', 'computer', 'our', 'spells.']

In [18]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [19]:
make_context_vector(data[0][0], word_to_ix)

tensor([21, 43, 26, 29])

In [20]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds_sum = embeds.sum(dim=0)
        out = F.relu(self.linear1(embeds_sum))
        out = self.linear2(out)
        out = out.view(1,-1)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [21]:
vocab_size = len(vocab)
embedding_dim = 10
context_size = CONTEXT_SIZE * 2

In [47]:
cbow = CBOW(vocab_size, embedding_dim, context_size)

In [52]:
inputs = make_context_vector(data[0][0], word_to_ix)

In [53]:
cbow.forward(inputs).shape

torch.Size([1, 49])

In [54]:
for context,target_word in data:
    inputs = make_context_vector(context, word_to_ix)
    target = torch.tensor([word_to_ix[target_word]], dtype=torch.long)

In [55]:
losses = []
loss_function = nn.NLLLoss()
optimizer = optim.SGD(cbow.parameters(), lr=0.001)

In [56]:
log_probs = cbow(inputs)

loss = loss_function(log_probs, target)

In [57]:
log_probs

tensor([[-4.0461, -4.0125, -4.3179, -4.2973, -3.6036, -4.2512, -3.7166, -4.4113,
         -4.0212, -4.0569, -3.9282, -4.4159, -3.9066, -4.2715, -3.9946, -4.2642,
         -4.2015, -4.8379, -3.9274, -3.9877, -4.2995, -3.8998, -3.4451, -3.8708,
         -4.1256, -3.6288, -4.1728, -4.2229, -4.0489, -3.6226, -3.8577, -3.6400,
         -3.5755, -4.2618, -3.1228, -4.3057, -4.2482, -4.0067, -3.4218, -3.8696,
         -4.0144, -3.7101, -4.3342, -3.3617, -3.7218, -4.1153, -3.0098, -4.0559,
         -3.4755]], grad_fn=<LogSoftmaxBackward>)

In [58]:
target

tensor([23])

In [59]:
losses = []
for epoch in range(50):
    total_loss = 0
    cbow.zero_grad()
    for idx,context,target_word in enumerate(data):
        
        inputs = make_context_vector(context, word_to_ix)
        target = torch.tensor([word_to_ix[target_word]], dtype=torch.long)
        
        
        log_probs = cbow(inputs)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(round(total_loss,2))
print(losses)

[227.41, 223.01, 218.79, 214.76, 210.9, 207.2, 203.66, 200.27, 197.01, 193.86, 190.82, 187.88, 185.02, 182.24, 179.53, 176.88, 174.3, 171.76, 169.28, 166.84, 164.44, 162.09, 159.77, 157.49, 155.25, 153.04, 150.86, 148.72, 146.6, 144.51, 142.46, 140.43, 138.42, 136.45, 134.5, 132.57, 130.68, 128.8, 126.96, 125.13, 123.33, 121.56, 119.8, 118.08, 116.38, 114.7, 113.04, 111.41, 109.8, 108.21]


In [33]:
vocab_size

49

In [50]:
save_input = make_context_vector(context, word_to_ix)

In [51]:
cbow.embeddings(save_input)

tensor([[ 1.7366,  0.5206, -0.1557, -0.1505,  0.2128,  0.3253,  0.0320, -0.6764,
          0.5815, -0.0758],
        [-0.1015,  0.2302, -3.5316, -0.7906, -0.0387, -0.7210, -1.4606, -0.6618,
         -0.4366,  0.2246],
        [ 1.5520, -1.4909,  2.7002,  0.0635,  0.4705, -0.9078,  1.2206, -0.0536,
          0.2215, -0.8316],
        [ 0.6551,  1.6814,  1.4042,  2.0463, -1.2023,  0.7464, -1.3545,  0.1785,
         -1.1858, -0.1496]], grad_fn=<EmbeddingBackward>)

In [60]:
cbow.embeddings(save_input)

tensor([[ 1.7537,  0.5214, -0.1930, -0.1318,  0.2242,  0.3212,  0.0504, -0.7063,
          0.5781, -0.0527],
        [-0.1032,  0.2283, -3.5441, -0.7801, -0.0457, -0.7263, -1.4684, -0.6612,
         -0.4442,  0.2281],
        [ 1.5608, -1.4858,  2.7010,  0.0690,  0.4701, -0.9181,  1.2226, -0.0540,
          0.2255, -0.8263],
        [ 0.6599,  1.6809,  1.4047,  2.0494, -1.2029,  0.7408, -1.3545,  0.1786,
         -1.1875, -0.1428]], grad_fn=<EmbeddingBackward>)

In [40]:
make_context_vector(context, word_to_ix)

tensor([ 4,  8, 19, 20])

In [229]:
for context,target_word in data:
    inputs = make_context_vector(context, word_to_ix)
    target = torch.tensor([word_to_ix[target_word]], dtype=torch.long)
    cbow.zero_grad()

    log_probs = cbow(inputs)
    
    argmax = int(log_probs.argmax())
    prediction = ix_to_word[argmax]
    
    print(f'{context[0]} {context[1]} [{target_word}] {context[2]} {context[3]} ==> {prediction}')

We are [about] to study ==> about
are about [to] study the ==> to
about to [study] the idea ==> study
to study [the] idea of ==> the
study the [idea] of a ==> the
the idea [of] a computational ==> of
idea of [a] computational process. ==> a
of a [computational] process. Computational ==> computational
a computational [process.] Computational processes ==> process.
computational process. [Computational] processes are ==> Computational
process. Computational [processes] are abstract ==> processes
Computational processes [are] abstract beings ==> are
processes are [abstract] beings that ==> abstract
are abstract [beings] that inhabit ==> manipulate
abstract beings [that] inhabit computers. ==> that
beings that [inhabit] computers. As ==> of
that inhabit [computers.] As they ==> a
inhabit computers. [As] they evolve, ==> As
computers. As [they] evolve, processes ==> they
As they [evolve,] processes manipulate ==> evolve,
they evolve, [processes] manipulate other ==> processes
evolve, proce